In [20]:
import skrub
from pathlib import Path
import pandas as pd
import sys
import numpy as np 
import matplotlib.pyplot as plt

from sklearn.model_selection import cross_validate
from skrub import tabular_learner
from skrub import TableVectorizer
from datetime import datetime
import os
import re 

sys.path.append('/Users/alina/Desktop/MIT/code/ADHD/MTA/helper')
from helper import rr, prep, var_dict, audit, plot

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
if Path('/Volumes/Samsung_T5/MIT/mta').exists():
    data_root =     '/Volumes/Samsung_T5/MIT/mta'
    data_derived  = '/Volumes/Samsung_T5/MIT/mta/output/derived_data'
else: 
    data_root = '/Users/alina/Desktop/MIT/code/data'
    data_derived  = '/Users/alina/Desktop/MIT/code/data/output/derived_data'
#os.listdir(data_root)

In [22]:
dtype_dict = {"GUID": str, 'String': str, "Integer": int, "Float": float, "Date": 'datetime64[ns]'}
pd.set_option('future.no_silent_downcasting', True)

treat_group_file = 'treatment_groups.csv'
trt_names = pd.read_csv(Path(data_derived, treat_group_file))

In [173]:
thr_missing_vals  = 50

qsts = []
exclude_files_dict ={ 20:   ('schoolhx', # no columns left afetr pre-auditing threshold 20
                            "diag", # check what to do with that 
                            "debrief", # no columns left afetr pre-auditing threshold 20
                            "scid", # no columns left afetr pre-auditing threshold 20
                            "debrief14",  # data only at timepoint 14
                            'scapijus',   # no uselful information 
                            'health') , 
                     
                     
                     50:    ('schoolhx',# no columns left afetr pre-auditing threshold 50
                             "diag", # chieck what to to with that 
                             "debrief14", # data only at timepoint 14
                             'scapijus' )}#  no useful information 

info_path = Path(data_root, "files")
os.listdir(info_path)


['.DS_Store',
 'debrief01_definitions.csv',
 'debrief1401_definitions.csv',
 'original_data',
 'schoolhx01_definitions.csv',
 'diagpsx01_definitions.csv',
 'expect01_definitions.csv',
 'cisp_p01_definitions.csv',
 'health01_definitions.csv',
 'masc_p01_definitions.csv',
 'snap01_definitions.csv',
 'scapijus01_definitions.csv',
 'cbcl01_definitions-2.csv',
 'pcrc01_definitions.csv',
 'medse01_definitions.csv',
 'ssrs01_definitions.csv',
 'wiat_iiip201_definitions.csv',
 'demgr01_definitions.csv',
 'scid01_definitions.csv']

In [174]:

split_chars = "[0_]"
info_dict = {}

for item in os.listdir(info_path):
    if not (item.startswith(".") or item.startswith(exclude_files_dict[thr_missing_vals])) and not os.path.isdir(Path(info_path, item)):
        name = re.split(split_chars, item)[0]
        qsts.append(name)
        print(name)
        info_dict[name] = pd.read_csv(Path(info_path, item), usecols=['ElementName','DataType', 'ElementDescription'] )


debrief
expect
cisp
health
masc
snap
cbcl
pcrc
medse
ssrs
wiat
demgr
scid


In [175]:
os.listdir(info_path)

['.DS_Store',
 'debrief01_definitions.csv',
 'debrief1401_definitions.csv',
 'original_data',
 'schoolhx01_definitions.csv',
 'diagpsx01_definitions.csv',
 'expect01_definitions.csv',
 'cisp_p01_definitions.csv',
 'health01_definitions.csv',
 'masc_p01_definitions.csv',
 'snap01_definitions.csv',
 'scapijus01_definitions.csv',
 'cbcl01_definitions-2.csv',
 'pcrc01_definitions.csv',
 'medse01_definitions.csv',
 'ssrs01_definitions.csv',
 'wiat_iiip201_definitions.csv',
 'demgr01_definitions.csv',
 'scid01_definitions.csv']

In [176]:
var_to_description = dict()

for item in info_dict.values():
    var = item['ElementName']
    descr = item['ElementDescription']
    var_to_description.update(dict(zip(var, descr)))
    
var_to_description['simple_relation'] = 'Simplified relaionship variable'

In [177]:
var_to_description

{'subjectkey': 'The NDAR Global Unique Identifier (GUID) for research subject',
 'src_subject_id': "Subject ID how it's defined in lab/project",
 'interview_date': 'Date on which the interview/genetic test/sampling/imaging/biospecimen was completed. MM/DD/YYYY',
 'interview_age': 'Age in months at the time of the interview/test/sampling/imaging.',
 'sex': 'Sex of subject at birth',
 'days_baseline': 'Days since baseline',
 'assbdic': 'Assessment Point',
 'relationship': 'Relationship of respondent to individual',
 'actbdic': 'Active status',
 'site': 'Site',
 'sjtyp': 'Subject type',
 'd2dresp': 'Which of the following choices best describes the responses of the parent(s) to randomization?',
 'd2dliki': 'Which of the following reflects your judgment about the likelihood of this family adhering to the protocol and remaining committed to the study?',
 'respond_detail_oth_spec': 'Respondent details',
 'exp1': 'Suppose you ask your child to clean up his/her room. Will your child do it?',
 

In [178]:
data_dict = {}
for item in os.listdir(data_root):
    if not (item.startswith(".") or item.startswith(exclude_files_dict[thr_missing_vals])) and not os.path.isdir(Path(data_root, item)):
    
        name = re.split(split_chars, item)[0]
        if name in info_dict.keys():
            data_dict[name] = pd.read_csv(Path(data_root, item), delimiter='\t')

/var/folders/hf/frc4nxb532j7gqr3_36l6kmh0000gn/T/ipykernel_86952/95897975.py:7: DtypeWarning: Columns (0,1,2,6,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,76,77,78,79,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158) have mixed types. Specify dtype option on import or set low_memory=False.
  data_dict[name] = pd.read_csv(Path(data_root, item), delimiter='\t')
/var/folders/hf/frc4nxb532j7gqr3_36l6kmh0000gn/T/ipykernel_86952/95897975.py:7: DtypeWarning: Columns (0,1,2,5,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,5

In [179]:
data_dict['debrief14'].assbdic.unique()


KeyError: 'debrief14'

In [ ]:
assert data_dict.keys() == info_dict.keys()

### Convert to appropriate datatype as found in description file 

### Verify later all questions that have these number as options, to check not to remove a valid answer

In [ ]:
missing_vals_pre_dict = {
    "cbcl": [999, -999, 88, 77, 98, 99, 9],
    "snap": [-999, 88, 77, -99, 999],
    "scid": [999, -9, -8, -999, -99, 88, 77, -7, 9],
    "schoolhx": [999, -999, 'NS', 'NK'],
    "cisp": [-999, 5],  # 5: not applicable, don't know in some questions
    "expect": [-999, "NR"],
    "health": ["NR", 88, -99, 77, -999, 99, -8, -9, -7, -1, "NK", 998, -5],
    "ssrs": [999, -9],
    "wiat": [999, -999, 67, -888],
    "debrief": [-999, 'NR'],
    "medse": ['NR'],
    "pcrc": [-999, 'NR'],
    "scapijus": ['NR'],  # here 99: "other"
    "debrief14": ["NR"],
    "masc": [-9, 99, 88, 99999, -9, 77, -999, 999, 888],
    "diags1": [-999, 9, -1, -2, 'NR'], 
    "demgr" : [-999, 'NR']
}

missing_val_list = [audit.get_missing_str(val) for val in missing_vals_pre_dict.values()]

missing_vals_dict = dict(zip(missing_vals_pre_dict.keys(),missing_val_list))


In [ ]:

relationship_dict=  {
        16: 0,  # Self
        # Family members
        1: 1, 2: 1, 3: 1, 17: 1, 18: 1, 19: 1, 20: 1, 21: 1, 22: 1, 23: 1, 24: 1,
        25: 1, 26: 1, 28: 1, 31: 1, 32: 1, 33: 1, 34: 1, 36: 1, 37: 1, 38: 1,
        39: 1, 40: 1, 41: 1, 42: 1, 43: 1, 44: 1, 45: 1, 46: 1, 47: 1, 48: 1,
        49: 1, 50: 1, 52: 1, 55: 1, 56: 1, 59: 1, 60: 1, 62: 1, 63: 1, 64: 1,
        65: 1, 66: 1, 67: 1, 68: 1, 69: 1, 70: 1, 71: 1, 72: 1, 73: 1, 74: 1,
        75: 1, 76: 1, 77: 1, 78: 1, 79: 1, 80: 1, 81: 1, 83: 1, 85: 1, 86: 1,
        87: 1, 89: 1, 90: 1, 91: 1, 92: 1, 93: 1,
        # Teachers
        4: 2, 5: 2, 14: 2,
        # Professional helpers
        6: 3, 7: 3, 8: 3, 9: 3, 10: 3, 11: 3, 12: 3, 15: 3, 51: 3, 54: 3, 58: 3,
        88: 3, 95: 3,
        # Others
        61: 4, 53: 4, 57: 4, 82: 4, 84: 4, 94: 4,
        # Missing values
        -999: np.nan, 13: np.nan
    }

In [ ]:
data_dict['demgr'].race.unique()


array(['Race of study subject', nan, 'White',
       'American Indian/Alaska Native', 'Black or African American',
       'More than one race', 'Asian', 'Hawaiian or Pacific Islander'],
      dtype=object)

### Keep src_subject_id until everything is put together 

In [180]:
qsts_ids = {qst : qst +"01_id" for qst in qsts}
qsts_ids

{'debrief': 'debrief01_id',
 'expect': 'expect01_id',
 'cisp': 'cisp01_id',
 'health': 'health01_id',
 'masc': 'masc01_id',
 'snap': 'snap01_id',
 'cbcl': 'cbcl01_id',
 'pcrc': 'pcrc01_id',
 'medse': 'medse01_id',
 'ssrs': 'ssrs01_id',
 'wiat': 'wiat01_id',
 'demgr': 'demgr01_id',
 'scid': 'scid01_id'}

In [181]:
cols_known2rem_dict = {
    "all": ['dataset_id', 'collection_id', 'subjectkey', 'sjtyp', 'interview_date', 
             'site', 'dataset_id', 'days_baseline'],
    #,  'assbdic' assessment point , 'actbdic' : active status ], # check there is 3rd value of type assbdic
    #'relationship',
    "snap": ['snaxrsp'],
    "cbcl": [],
    "scid": [],
    "schoolhx": ['schoolhx01_id'],
    "cisp": ['comments_misc'],  # miscellaneous comments
    "expect": [],
    "health": [],
    "ssrs": [],
    "wiat": [],
    "debrief": [],
    "medse": [],
    "pcrc": [],
    "scapijus":  [], # here 99: "other"
    "debrief14":[],
    "masc": [],
    "diags1": [],
    "demgr" : []
}

for qst in qsts:
    cols_known2rem_dict.setdefault(qst,[]).append(qsts_ids[qst])
    cols_known2rem_dict[qst].extend(cols_known2rem_dict["all"])
    
const_cols2keep = ['relationship', 'assbdic', 'actbdic']

#cols_known2rem_dict['2keep'] = const_cols2keep

In [182]:
categorical_columns = data_dict['ssrs'].select_dtypes(include=['category']).columns.tolist()
categorical_columns

[]

In [183]:
test = audit.pre_audit(  data_dict["snap"], info_dict["snap"], missing_vals_dict['snap'], dtype_dict, cols_known2rem_dict['snap'], cols_known_to_keep=const_cols2keep)

original shape :  (14581, 160)
Removing subject_unspecific columns ..  N =  5
(14580, 155)
Setting dtypes..
Index(['respondent', 'respondent_other_specify', 'snap_adhd_10',
       'snap_adhd_20', 'snap_inattn_totalscore', 'snap_inattn_avg',
       'snap_hyp_totalscore', 'snap_hyp_avg', 'daysrz', 'study', 'completed',
       'subtrial_name', 'visit', 'resp_sex', 'snap_iv_pac36', 'snap_iv_pac37',
       'snap_iv_pac38', 'snap_iv_pac39', 'snap_iv_pac40', 'snap_iv_pac41',
       'snap_iv_pac42', 'snap_iv_pac43', 'snap_iv_pac44', 'snap_iv_pac45',
       'snap_iv_pac46', 'snap_iv_pac47', 'snap_iv_pac48', 'snap_iv_pac49',
       'snap_iv_pac50', 'snap_iv_pac51', 'snap_iv_pac52', 'snap_iv_pac53',
       'snap_iv_pac54', 'snap_iv_pac55', 'snap_iv_pac29', 'snap_iv_pac56',
       'snap_iv_pac57', 'snap_iv_pac58', 'snap_iv_pac59', 'snap_iv_pac60',
       'snap_iv_pac61', 'snap_iv_pac62', 'snap_iv_pac63', 'snap_iv_pac64',
       'snap_iv_pac65', 'snap_iv_pac30', 'snap_iv_pac66', 'snap_iv_pac67',
  

In [184]:
cols_known2rem_dict.keys()

dict_keys(['all', 'snap', 'cbcl', 'scid', 'schoolhx', 'cisp', 'expect', 'health', 'ssrs', 'wiat', 'debrief', 'medse', 'pcrc', 'scapijus', 'debrief14', 'masc', 'diags1', 'demgr'])

In [185]:
data_dict_preaudited = {}
for qst in qsts:
    print(qst)
    data_dict_preaudited[qst] = audit.pre_audit(  data_dict[qst], info_dict[qst], missing_vals_dict[qst], dtype_dict, cols_known2rem_dict[qst], cols_known_to_keep=const_cols2keep,  thr_drop_missing =  thr_missing_vals)

debrief
original shape :  (580, 18)
Removing subject_unspecific columns ..  N =  5
(579, 13)
Setting dtypes..
Index([], dtype='object')
Removing empty columns ..  N =  0
(579, 13)
Removing constant columns .. N =  2
(579, 11)
Removing known and raw columns..  N =  : 8
    ERROR
"['debrief01_id', 'dataset_id', 'sjtyp', 'days_baseline', 'collection_id', 'interview_date'] not found in axis"
Removing non-existing columns from list .. 
Success. Removing known and raw columns..  N =  : 2
(579, 9)
Removing above threshold empty columns.. N =  : 1
(579, 8)
Old shape:  (580, 18)
New shape:  (579, 8)
Remaining, ends with x :  ['sex']
Remaining, ends with t :  []
Remaining, ends with raw :  []
src_subject_id     object
interview_age       int64
sex                object
assbdic            object
relationship      float64
actbdic             int64
d2dresp           float64
d2dliki           float64
dtype: object


expect
original shape :  (5069, 23)
Removing subject_unspecific columns ..  N =  5
(

In [186]:
data_dict_preaudited.keys()

dict_keys(['debrief', 'expect', 'cisp', 'health', 'masc', 'snap', 'cbcl', 'pcrc', 'medse', 'ssrs', 'wiat', 'demgr', 'scid'])

In [187]:
table_report_all = False
for qst in qsts:
    if table_report_all:
    
        skrub.TableReport(data_dict_preaudited[qst]).open()
    print(qst, data_dict_preaudited[qst].shape, 'relationship' in data_dict_preaudited[qst].columns)
        

debrief (579, 8) True
expect (5068, 13) True
cisp (3988, 26) True
health (2827, 147) True
masc (4703, 71) True
snap (14580, 56) True
cbcl (4063, 32) True
pcrc (8712, 71) True
medse (3431, 29) False
ssrs (11038, 100) True
wiat (6243, 10) True
demgr (4076, 23) True
scid (1641, 81) True


## Map relationship

In [188]:
child_questionaires = ['masc', 'wiat']

no_relationship =  [qst for qst in qsts if 'relationship' not in data_dict_preaudited[qst].columns]
qsts_relationship = list(filter(lambda x: x not in no_relationship, qsts))
print(no_relationship)

['medse']


In [189]:
for qst in qsts_relationship:
        data_dict_preaudited[qst].loc[:, 'simple_relation'] =  data_dict_preaudited[qst]['relationship'].map(relationship_dict)

## Get Baseline

### check why there is no assessment or days baseline or relationship variable in medse

### Demogr \ Cisp has no baseline values ?  --> ask Dr Surman

In [190]:
data_dict_preaudited['debrief'].assbdic.unique()

array(['D'], dtype=object)

In [191]:
qst_assbdic = [qst for qst in qsts if 'assbdic' in data_dict_preaudited[qst].columns]
qst_assbdic
qst_no_assbdic = list(filter(lambda x: x not in qst_assbdic, qsts))
qst_no_assbdic


['medse']

In [192]:
baseline_data_prea = {}
for qst in qst_assbdic:
        baseline_data_prea[qst] =  data_dict_preaudited[qst][data_dict_preaudited[qst]['assbdic'] == 'D']
        print(qst , baseline_data_prea[qst].shape)
        #print((baseline_data_prea[qst].src_subject_id.value_counts()> 1).sum())

debrief (579, 9)
expect (917, 14)
cisp (0, 27)
health (0, 148)
masc (575, 72)
snap (1464, 57)
cbcl (927, 33)
pcrc (1927, 72)
ssrs (1475, 101)
wiat (579, 11)
demgr (0, 24)
scid (1277, 82)


In [198]:
qsts_empty_baseline = [qst for qst in qst_assbdic if baseline_data_prea[qst].shape[0] == 0]
qsts_baseline = list(filter(lambda x: x not in qsts_empty_baseline, qst_assbdic))

#### EVERYBODY ACTIVE -> check when add new qsts 

In [206]:
for qst in qsts_baseline:
    assert baseline_data_prea[qst].actbdic.unique() == 1

In [105]:
drop_cols_single = ['sex', 'interview_age']
drop_cols_all = [ 'assbdic',  'actbdic'] # 'relationship',

drop_cols_other = ['version_form'] # check again which ones 



In [106]:
test_ = pd.merge(baseline_data_prea['ssrs'][baseline_data_prea['ssrs']['simple_relation'] == 1].drop(columns = drop_cols_all), baseline_data_prea['snap'][baseline_data_prea['snap']['simple_relation'] == 1].drop(columns= np.concatenate((drop_cols_all, drop_cols_single)), inplace = False), on=['src_subject_id', 'relationship'])

In [102]:
for qst in qsts_relationship :
    print(baseline_data_prea[qst].simple_relation.value_counts()
          )

simple_relation
1.0    264
Name: count, dtype: int64
simple_relation
1.0    917
Name: count, dtype: int64
Series([], Name: count, dtype: int64)
Series([], Name: count, dtype: int64)
simple_relation
0.0    575
Name: count, dtype: int64
simple_relation
1.0    911
2.0    546
Name: count, dtype: int64
simple_relation
1.0    927
Name: count, dtype: int64
simple_relation
1.0    1927
Name: count, dtype: int64
simple_relation
1.0    929
2.0    546
Name: count, dtype: int64
simple_relation
0.0    579
Name: count, dtype: int64
Series([], Name: count, dtype: int64)
simple_relation
1.0    1277
Name: count, dtype: int64


In [103]:
for qst in qsts_relationship:
    print(f"Question: {qst}")
    
    # Group by subject_id and relationship, and count occurrences
    relationship_counts = (
        baseline_data_prea[qst]
        .groupby(['src_subject_id', 'relationship'])
        .size()
        .unstack(fill_value=0)
    )
    
    # Print the counts for each subject
    for subject_id, counts in relationship_counts.iterrows():
        print(f"Subject {subject_id}:")
        for relationship, count in counts.items():
            if count > 1:
                print(f"  Relationship {relationship}: {count}")
        print("-" * 20)

Question: debrief
Subject P1023:
--------------------
Subject P1024:
--------------------
Subject P1034:
--------------------
Subject P1039:
--------------------
Subject P1040:
--------------------
Subject P1043:
--------------------
Subject P1046:
--------------------
Subject P1048:
--------------------
Subject P1050:
--------------------
Subject P1052:
--------------------
Subject P1060:
--------------------
Subject P1062:
--------------------
Subject P1066:
--------------------
Subject P1076:
--------------------
Subject P1078:
--------------------
Subject P1079:
--------------------
Subject P1081:
--------------------
Subject P1085:
--------------------
Subject P1086:
--------------------
Subject P1092:
--------------------
Subject P1093:
--------------------
Subject P1094:
--------------------
Subject P1102:
--------------------
Subject P1103:
--------------------
Subject P1107:
--------------------
Subject P1110:
--------------------
Subject P1111:
--------------------
Subject P1

## Option1 : remove  subject with 2 answers per subject 

In [ ]:
relationship_counts = (
    test_
    .groupby(['src_subject_id', 'relationship'])
    .size()
    .unstack(fill_value=0)
)

# Print the counts for each subject
for subject_id, counts in relationship_counts.iterrows():
    print(f"Subject {subject_id}:")
    for relationship, count in counts.items():
        if count > 0:
            print(f"  Relationship {relationship}: {count}")
    print("-" * 20)

Subject P1002:
  Relationship 1: 1
  Relationship 2: 1
--------------------
Subject P1003:
  Relationship 1: 1
  Relationship 2: 1
--------------------
Subject P1004:
  Relationship 1: 1
--------------------
Subject P1005:
  Relationship 21: 1
--------------------
Subject P1006:
  Relationship 1: 1
  Relationship 2: 1
--------------------
Subject P1007:
  Relationship 1: 1
  Relationship 2: 1
--------------------
Subject P1008:
  Relationship 1: 1
  Relationship 21: 1
--------------------
Subject P1009:
  Relationship 1: 1
  Relationship 2: 1
--------------------
Subject P1010:
  Relationship 1: 1
--------------------
Subject P1011:
  Relationship 1: 1
--------------------
Subject P1013:
  Relationship 1: 1
  Relationship 2: 1
--------------------
Subject P1014:
  Relationship 1: 1
--------------------
Subject P1015:
  Relationship 1: 1
  Relationship 2: 1
--------------------
Subject P1017:
  Relationship 1: 1
--------------------
Subject P1019:
  Relationship 1: 1
-------------------

In [107]:
baseline_data_prea_test = baseline_data_prea.copy()

In [108]:
test_snap = baseline_data_prea_test['snap'][(baseline_data_prea_test['snap']['relationship'] == 1 ) ]
(test_snap.src_subject_id.value_counts() > 1).sum()

0

In [116]:
base_c_prea_drop = {}

base_p_prea_drop = {}
base_t_prea_drop = {}

In [119]:
for qst in qsts_relationship:
    base_c_prea_drop[qst] = baseline_data_prea[qst][(baseline_data_prea[qst]['simple_relation'] == 0) ]
    base_p_prea_drop[qst] = baseline_data_prea[qst][(baseline_data_prea[qst]['simple_relation'] == 1 ) ]
    base_t_prea_drop[qst] = baseline_data_prea[qst][(baseline_data_prea[qst]['simple_relation'] == 2 ) ]

In [121]:
for qst in qsts_relationship:
    print(qst, 'c',base_c_prea_drop[qst].shape, (base_c_prea_drop[qst].src_subject_id.value_counts() > 1).sum() )
    print(qst, 'p',base_p_prea_drop[qst].shape, (base_p_prea_drop[qst].src_subject_id.value_counts() > 1).sum() )
    print(qst, 't',base_t_prea_drop[qst].shape,  (base_t_prea_drop[qst].src_subject_id.value_counts() > 1).sum() )
    

debrief c (0, 9) 0
debrief p (264, 9) 0
debrief t (0, 9) 0
expect c (0, 14) 0
expect p (917, 14) 347
expect t (0, 14) 0
cisp c (0, 27) 0
cisp p (0, 27) 0
cisp t (0, 27) 0
health c (0, 148) 0
health p (0, 148) 0
health t (0, 148) 0
masc c (575, 72) 0
masc p (0, 72) 0
masc t (0, 72) 0
snap c (0, 57) 0
snap p (911, 57) 342
snap t (546, 57) 1
cbcl c (0, 33) 0
cbcl p (927, 33) 354
cbcl t (0, 33) 0
pcrc c (0, 72) 0
pcrc p (1927, 72) 573
pcrc t (0, 72) 0
ssrs c (0, 101) 0
ssrs p (929, 101) 353
ssrs t (546, 101) 0
wiat c (579, 11) 0
wiat p (0, 11) 0
wiat t (0, 11) 0
demgr c (0, 24) 0
demgr p (0, 24) 0
demgr t (0, 24) 0
scid c (0, 82) 0
scid p (1277, 82) 424
scid t (0, 82) 0


### Remove all duplicate subjects 

In [112]:
def remove_duplicates(df, col_to_check, keep = False): # keep first occurance of duplicate if set to True
    df = df.loc[~df[col_to_check].duplicated(keep=keep), :]
    return df
    

In [113]:
for qst in qsts_relationship:
    base_p_prea_drop[qst] = remove_duplicates(base_p_prea_drop[qst], 'src_subject_id', keep = False)
    base_t_prea_drop[qst] = remove_duplicates(base_t_prea_drop[qst], 'src_subject_id', keep = False)


In [114]:
for qst in qsts:
    if qst in qsts_relationship:
        print(qst, 'p',base_p_prea_drop[qst].shape, (base_p_prea_drop[qst].src_subject_id.value_counts() > 1).sum() )
        print(qst, 't',base_t_prea_drop[qst].shape,  (base_t_prea_drop[qst].src_subject_id.value_counts() > 1).sum() )
    else:
        print(qst,  baseline_data_prea[qst].shape, (baseline_data_prea[qst].src_subject_id.value_counts() > 1).sum() )

debrief p (264, 9) 0
debrief t (0, 9) 0
expect p (215, 14) 0
expect t (0, 14) 0
health p (0, 148) 0
health t (0, 148) 0
masc p (0, 72) 0
masc t (0, 72) 0
snap p (219, 57) 0
snap t (544, 57) 0
cbcl p (215, 33) 0
cbcl t (0, 33) 0
pcrc p (6, 72) 0
pcrc t (0, 72) 0
ssrs p (219, 101) 0
ssrs t (546, 101) 0
wiat p (0, 11) 0
wiat t (0, 11) 0
scid p (154, 82) 0
scid t (0, 82) 0


### Merge parent and teacher

In [ ]:
base_p_t_drop = {}

In [ ]:
for qst in qsts_relationship:
    df = pd.merge(base_p_prea_drop[qst], base_t_prea_drop[qst], on= 'src_subject_id', how = 'outer', suffixes=('_p', '_t'))
    df = df.dropna(axis=1, how='all')
    base_p_t_drop[qst] = df

for qst in no_relationship:
    base_p_t_drop[qst] = baseline_data_prea[qst]
    
    

,src_subject_id,interview_age,sex,masc_p_1,masc_p_2,masc_p_3,masc_p_4,masc_p_5,masc_p_6,masc_p_7,...,ma12sax,ma21px,ma22acx,ma31hfx,ma32pfx,ma1phyx,ma2hat,ma3socx,ma4sepx,matotalx
4,P1002,120.0,M,2.0,2.0,2.0,2.0,3.0,2.0,2.0,...,2.67,3.0,3.00,3.17,2.83,2.75,33.0,3.00,2.9,2.91
8,P1003,108.0,M,2.0,3.0,3.0,2.0,2.0,2.0,2.0,...,2.50,3.6,3.83,3.00,3.33,2.58,41.0,3.17,2.4,2.98
14,P1004,108.0,F,1.0,3.0,1.0,1.0,2.0,0.0,1.0,...,2.00,3.2,2.33,1.80,2.17,1.83,30.0,2.00,2.0,2.14
21,P1005,108.0,M,2.0,3.0,3.0,3.0,1.0,0.0,0.0,...,2.17,3.6,2.00,3.00,4.00,2.00,30.0,3.50,2.2,2.62
26,P1006,108.0,M,0.0,3.0,3.0,2.0,3.0,1.0,2.0,...,3.00,4.0,4.00,3.50,3.50,3.17,44.0,3.50,3.3,3.49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4505,P1830,84.0,M,1.0,3.0,1.0,1.0,1.0,1.0,3.0,...,3.00,3.2,2.67,2.67,1.83,2.67,32.0,2.25,2.4,2.56
4543,P1837,72.0,M,0.0,3.0,0.0,0.0,3.0,0.0,0.0,...,1.00,3.4,3.00,1.50,2.00,1.25,35.0,1.75,2.5,2.13
4555,P1839,72.0,M,2.0,3.0,1.0,3.0,0.0,3.0,3.0,...,1.50,3.4,2.50,1.17,2.00,1.67,32.0,1.58,2.5,2.13
4573,P1842,72.0,M,1.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.00,4.0,3.50,3.50,4.00,3.33,41.0,3.75,2.5,3.36


### Multiple sample relationship answers
option 1 : drop the subject 
option 2 : average the answers

In [ ]:
for qst in qsts:
    print(   'QST : ', qst)
    for var in data_dict_preaudited[qst].columns:
        print(var, var_to_description[var],  data_dict_preaudited[qst][var].dropna().shape)
    print('\n')


### Inspection (to remove)

This is data from month 14 only anyways 
- debrief14  (all): 
  - eotcat End of treatment category (431,) # dont remove for potential change of group info 
check 
  - Number of recommendations given at debriefing (399,)
  

health 
- lots of past two year diagnosis --> compare to other diagnostic files 

masc
- none

snap
 - none 

scapijus 
delete all (no info):
src_subject_id Subject ID how it's defined in lab/project (27308,)
interview_age Age in months at the time of the interview/test/sampling/imaging. (20393,)
sex Sex of subject at birth (27308,)
monthsbl Actual number of months from Baseline (23253,)
daystrt Parent SNAP or CAARS Days from Baseline used as start of interval for SCAPI data (20393,)
dayend Parent SNAP or CAARS Days from Baseline used as end of interval for SCAPI data (23252,)
snapyn Is there a Parent SNAP or CAARS data-form present for the end-of-interval date? (20373,)
knownpc Percentage of the interval for which data are known (27158,)
intlnth Interval length (20254,)
version_form Form used/assessment name (27308,)
asssca Assessment point (24360,)

cbcl: 
- none 

pc: 
- maybe version-form 

medse
This is a information over time qsts 
medication visit number 
compliance, some symptoms, side effects, dosage regiment 

ssrs :
- none 

wiat : 
- maybe : atest Achievement test administered (6243,)

demgr:
- none 

scid 
- lots of diagnoses too -- but not enough data if cutoff drop column is 20% -- chekc when only baseline data is considered 







  


In [ ]:
for qst in qsts:
    print(   'QST : ', qst)
    for var in data_dict_preaudited[qst].columns:
        if data_dict_preaudited[qst][var].dtype == object:
            print(var, var_to_description[var],  data_dict_preaudited[qst][var].dropna().shape)
    print('\n')

In [ ]:
# for qst in qsts:
#     print(   'QST : ', qst)
#     print(baseline_data_prea[qst].assbdic.unique())
#     try : 
#         print('RELATIONSHIP ', baseline_data_prea[qst].relationship.unique())
#     except AttributeError:
#         print('No relationship var'.upper())
#     try: 
#         print('VERSION FORM ', baseline_data_prea[qst].version_form.unique())
#     except AttributeError:
#         print('No version form'.upper())
    
#     print('\n')

### Check if there is more than 1 entry per category --> remove

In [ ]:
baseline_data_prea['cbcl']

,src_subject_id,interview_age,sex,sports_time1,sports_well1,sports_time2,sports_well2,activities_time1,activities_well1,chores_well1,...,school1b,school1c,school1d,special_ed,repeat_grade,academic,relationship,assbdic,actbdic,simple_relation
5,P1002,120.0,M,1.0,1.0,1.0,2.0,1.0,2.0,1.0,...,2.0,2.0,2.0,1.0,0.0,1.0,1,D,1,1.0
6,P1002,120.0,M,1.0,1.0,2.0,1.0,1.0,1.0,0.0,...,3.0,2.0,3.0,1.0,0.0,1.0,2,D,1,1.0
11,P1003,108.0,M,1.0,1.0,2.0,1.0,1.0,2.0,0.0,...,2.0,2.0,1.0,1.0,1.0,1.0,1,D,1,1.0
12,P1003,108.0,M,2.0,1.0,1.0,1.0,1.0,2.0,1.0,...,1.0,1.0,1.0,0.0,1.0,1.0,2,D,1,1.0
19,P1004,108.0,F,1.0,1.0,NaN,NaN,1.0,1.0,1.0,...,2.0,1.0,1.0,0.0,0.0,0.0,1,D,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3959,P1839,72.0,M,1.0,3.0,1.0,2.0,2.0,1.0,0.0,...,2.0,2.0,2.0,0.0,0.0,1.0,1,D,1,1.0
3960,P1839,72.0,M,0.0,2.0,0.0,1.0,2.0,2.0,2.0,...,NaN,NaN,NaN,0.0,0.0,1.0,2,D,1,1.0
3973,P1842,72.0,M,2.0,2.0,1.0,2.0,2.0,2.0,0.0,...,2.0,2.0,3.0,0.0,0.0,1.0,1,D,1,1.0
3980,P1844,72.0,M,0.0,1.0,2.0,2.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1,D,1,1.0


In [ ]:
test_relationship_snap = baseline_data_prea['snap'][['src_subject_id', 'relationship']]

In [ ]:
test_relationship_snap.loc[:, 'simple_relation'] = baseline_data_prea['snap']['relationship'].map(relationship_dict)

/var/folders/hf/frc4nxb532j7gqr3_36l6kmh0000gn/T/ipykernel_83925/3833292943.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_relationship_snap.loc[:, 'simple_relation'] = baseline_data_prea['snap']['relationship'].map(relationship_dict)


In [ ]:
data_dict['cisp'].relationship

0       Relationship of respondent to individual
1                                              1
2                                              1
3                                              1
4                                              1
                          ...                   
3984                                           1
3985                                           1
3986                                           1
3987                                           1
3988                                           1
Name: relationship, Length: 3989, dtype: object